In [1]:
%load_ext dotenv
%dotenv

In [2]:
# !pip install --upgrade langchain langchain-community langchain-google-genai 
# !pip install -U langchain-groq

In [3]:
from ipywidgets import widgets, interact, interactive, Layout
from IPython.display import display, Markdown
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_groq import ChatGroq  
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [14]:
class LLMProvider:
    def __init__(self, provider: str, model: str):
        """
        Initialize the LLMProvider class.

        Args:
            provider (str): The provider of the LLM.
            model (str): The model of the LLM.
        """
        self.provider = provider
        self.model = model

    def initialize_llm(self) -> object:
        """
        Initialize the LLM.

        Returns:
            object: The initialized LLM.
        """
        try:
            if self.provider == "google":
                return GoogleGenerativeAI(model=self.model)
            elif self.provider == "groq":
                return ChatGroq(model=self.model)
            else:
                raise ValueError("Unsupported LLM provider")
        except Exception as e:
            print(f"Error initializing LLM: {e}")
            return None

class LLMAPI:
    def __init__(self, provider: str, model: str):
        """
        Initialize the LLMAPI class.

        Args:
            provider (str): The provider of the LLM.
            model (str): The model of the LLM.
            api_key (str): The API key for the LLM.
        """
        self.provider = LLMProvider(provider, model)
        self.llm = self.provider.initialize_llm()
        self.memory = ConversationBufferMemory(memory_key="history", return_messages=True)
        self.conversation_chain = ConversationChain(llm=self.llm, memory=self.memory)

    def get_response(self, user_message: str) -> str:
        """
        Get the response from the LLM.

        Args:
            user_message (str): The user's message.

        Returns:
            str: The response from the LLM.
        """
        try:
            response = self.conversation_chain.invoke(input=user_message)
            return response['response'].strip()
        except Exception as e:
            print(f"Error getting response: {e}")
            return ""


# Initialize the LLM API
llm_api = LLMAPI(provider="groq", model="llama-3.1-70b-versatile")

# Create the chat area
chat_area = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow': 'auto'})

# Create the input field with a larger layout
input_field = widgets.Textarea(placeholder='Type your message here...',
                              layout=Layout(width='100%', height='80px'))

# Create the send button
send_button = widgets.Button(description='Send')

# Arrange the widgets in a layout
layout = widgets.VBox([chat_area, widgets.HBox([input_field, send_button])])

def send_message(sender):
    user_message = input_field.value
    input_field.value = ''
    
    with chat_area:
        msg = f'**You:** {user_message}'
        display(Markdown(msg))
    
    agent_response = llm_api.get_response(user_message)
    
    with chat_area:
        msg = f'**Agent:** {agent_response}'
        display(Markdown(msg))

send_button.on_click(send_message)

display(layout)